In [52]:
import requests, json
from urllib.parse import quote
import folium
import pandas as pd
import numpy as np

In [53]:
filename = 'data/경기도_geo_simple.json'
geo_data = json.load(open(filename, encoding='utf-8'))

In [54]:
area = pd.read_csv('data/전국도시공원정보표준데이터.csv',encoding='euc-kr')
area.head()

,관리번호,공원명,공원구분,소재지도로명주소,소재지지번주소,위도,경도,공원면적,공원보유시설(운동시설),공원보유시설(유희시설),공원보유시설(편익시설),공원보유시설(교양시설),공원보유시설(기타시설),지정고시일,관리기관명,전화번호,데이터기준일자,제공기관코드,제공기관명
0,11170-00048,녹색소공원,소공원,NaN,서울특별시 용산구 효창동 287-6,37.539217,126.962694,1569.9,NaN,NaN,NaN,NaN,NaN,2012-10-19,서울특별시 용산구청,02-2199-7605,2022-12-20,3020000,서울특별시 용산구
1,36110-00058,푸름샘 소공원(소1-1),소공원,세종특별자치시 마음안로 161,NaN,36.516117,127.233781,1820.0,NaN,NaN,앉음벽,NaN,실개천,2011-08-05,세종특별자치시 공원관리사업소,044-301-3613,2022-12-14,5690000,세종특별자치시
2,36110-00059,솔숲 소공원(소1-2),소공원,세종특별자치시 만남1북로 73,NaN,36.508995,127.230040,1037.0,NaN,조합놀이대+ 그네+ 시소+ 흔들놀이,의자+ 파고라,NaN,안내판,2011-08-05,세종특별자치시 공원관리사업소,044-301-3613,2022-12-14,5690000,세종특별자치시
3,36110-00060,마디꽃 수변공원(수1-1),수변공원,세종특별자치시 다솜로 38,NaN,36.495962,127.257316,4928.0,NaN,NaN,정자+ 의자,NaN,NaN,2007-07-04,세종특별자치시 공원관리사업소,044-301-3633,2022-12-14,5690000,세종특별자치시
4,36110-00061,꽃 수변공원(수2-3),수변공원,세종특별자치시 나리로 26-10,NaN,36.473133,127.254482,3575.0,배드민터장+ 운동기구3종,NaN,자전거보관대+ 평의자+ 안내판,NaN,NaN,2009-01-07,세종특별자치시 공원관리사업소,044-301-3613,2022-12-14,5690000,세종특별자치시


In [55]:
area = area[['공원명','소재지지번주소','공원면적']]

In [56]:
area.dropna(subset=['소재지지번주소'], inplace=True)
area.isna().sum().sum()

0

In [57]:
area.head()

,공원명,소재지지번주소,공원면적
0,녹색소공원,서울특별시 용산구 효창동 287-6,1569.9
23,강변공원,서울특별시 용산구 이촌동 408,1766.0
24,솔밭어린이공원,서울특별시 용산구 한강로1가 30-3,2090.0
25,신창동어린이공원,서울특별시 용산구 신창동 100,257.0
26,서빙고공원,서울특별시 용산구 용산동5가 24-2,11879.0


In [58]:
area = area[area['소재지지번주소'].str.contains('경기도')]
area.head()

,공원명,소재지지번주소,공원면적
212,꿈나무공원,경기도 이천시 중리동 15,2526.0
213,남천공원,경기도 이천시 중리동 210,3830.0
214,향교공원,경기도 이천시 창전동 408,1757.0
215,개나리공원,경기도 이천시 창전동 414-13,1650.0
216,달맞이공원,경기도 이천시 창전동 423,1650.0


In [78]:
area['시군구'] = area.소재지지번주소.apply(lambda x: ''.join(x.split()[1]))
area

,공원명,소재지지번주소,공원면적,시군구
시군구,,,,
이천시,꿈나무공원,경기도 이천시 중리동 15,2526.0,이천시
이천시,남천공원,경기도 이천시 중리동 210,3830.0,이천시
이천시,향교공원,경기도 이천시 창전동 408,1757.0,이천시
이천시,개나리공원,경기도 이천시 창전동 414-13,1650.0,이천시
이천시,달맞이공원,경기도 이천시 창전동 423,1650.0,이천시
...,...,...,...,...
용인시,제90호 어린이공원(풀향기),경기도 용인시 기흥구 중동 1083,1500.0,용인시
광주시,청석공원,경기도 광주시 경안동 242,193530.0,광주시
광주시,경안근린공원,경기도 광주시 경안동 168-1,83095.0,광주시


In [65]:
area.set_index('시군구', inplace=True)
area.head()

,공원명,소재지지번주소,공원면적
시군구,,,
이천시,꿈나무공원,경기도 이천시 중리동 15,2526.0
이천시,남천공원,경기도 이천시 중리동 210,3830.0
이천시,향교공원,경기도 이천시 창전동 408,1757.0
이천시,개나리공원,경기도 이천시 창전동 414-13,1650.0
이천시,달맞이공원,경기도 이천시 창전동 423,1650.0


In [66]:
map = folium.Map([37.4,127.1], zoom_start=9, tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_data,
    data=area.공원면적,
    columns=[area.index, area.공원면적],
    fill_color='YlOrRd',
    key_on='properties.name'
).add_to(map)
map